# Setup

In [ ]:
import cv2
import cv2.aruco as aruco
import os
import numpy as np
import subprocess

calib_id = "07-31-25"
calib_base_folder = f"/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/{calib_id}"
# calib_base_folder = f"C:\\Users\\jlim\\Documents\\GitHub\\Catheter-Perception\\camera_calibration\\{calib_id}"
# Camera 0 refers to top camera, Camera 1 refers to bottom camera
# Ensure camera port ids are correct
# port_ids = [0, 2]
cam0_device = "/dev/cam0"
cam1_device = "/dev/cam1"
cam2_device = "/dev/cam2"
devices = [f"/dev/cam{i}" for i in range(3)]
cap_frame_width = 1280
cap_frame_height = 720

# Ensure proper camera configurations
focus_values = [35, 75, 75]  # Default focus values for cam0, cam1, cam2
config_commands = {cam0_device: [
                    f"v4l2-ctl -d {cam0_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam0_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam0_device} -c focus_absolute={focus_values[0]}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                cam1_device: [
                    f"v4l2-ctl -d {cam1_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam1_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam1_device} -c focus_absolute={focus_values[1]}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                cam2_device: [
                    f"v4l2-ctl -d {cam2_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam2_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam2_device} -c focus_absolute={focus_values[2]}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                }

def configure_camera(devices, config_commands):
    for device in devices:

        print(f"Configuring camera on {device}...")

        for command in config_commands[device]:
            subprocess.run(command, shell=True, check=True)

        print("Camera configuration complete!")

# Find Camera Focus Value

In [ ]:
import subprocess
import cv2
import time

# Configure and open cameras to check proper configs
d=0
cap = cv2.VideoCapture(devices[d], cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)

if not cap.isOpened():
    print("Error: One or both cameras could not be opened.")
    exit()

focus_adjust = focus_values[d]
configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera([devices[d]], config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    ret, frame = cap.read()

    if not ret:
        print("Error: One or both frames could not be read.")
        break
    cv2.putText(frame, f"Focus: {focus_adjust}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(f"Camera {d}", frame)


    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord('q'):  # Increase focus for Camera 0
        focus_adjust = min(focus_adjust + 5, 250)
        command = f"v4l2-ctl -d {devices[d]} -c focus_absolute={focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam {d} focus set to: {focus_adjust}")
    elif key == ord('a'):  # Decrease focus for Camera 0
        focus_adjust = max(focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {devices[d]} -c focus_absolute={focus_adjust}"
        subprocess.run(command, shell=True)

# Release both cameras and close windows
cap.release()
cv2.destroyAllWindows()

# Check Camera configs

In [ ]:
import subprocess
import cv2
import time

# Configure and open cameras to check proper configs
caps = []
for device in devices:
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)
    caps.append(cap)
    if not cap.isOpened():
        print(f"Error: Camera {device} could not be opened.")
        exit()

configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera([cam0_device, cam1_device], config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    for i, cap in enumerate(caps):
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame could not be read from camera.")
            exit()
        cv2.imshow(f"Camera {i} (Device: {devices[i]})", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord('q'):  # Increase focus for Camera 0
        cam0 = min(cam0 + 5, 250)
        command = f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 0 focus set to: {cam0_focus_adjust}")
    elif key == ord('a'):  # Decrease focus for Camera 0
        cam0_focus_adjust = max(cam0_focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 0 focus set to: {cam0_focus_adjust}")
    elif key == ord('w'): # Increase focus for Camera 1
        cam1_focus_adjust = min(cam1_focus_adjust + 5, 250)
        command = f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 1 focus set to: {cam1_focus_adjust}")
    elif key == ord('s'):
        # Decrease focus for Camera 1
        cam1_focus_adjust = max(cam1_focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 1 focus set to: {cam1_focus_adjust}")

# Release both cameras and close windows
for cap in caps:
    cap.release()
cv2.destroyAllWindows()

# Crop ROI

In [ ]:
import cv2

roi_boxes = []
for d in range(len(devices)):
    cap = cv2.VideoCapture(devices[d], cv2.CAP_V4L2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)

    print("Press SPACE to freeze frame and select ROI.")
    print("Draw the box, then press ENTER to confirm or ESC to cancel.")

    roi_box = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        cv2.imshow("Live Feed", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == 32:  # SPACE to pause and select ROI
            cv2.imshow("Select ROI", frame)
            roi_box = cv2.selectROI("Select ROI", frame, showCrosshair=True, fromCenter=False)
            cv2.destroyWindow("Select ROI")

            if roi_box != (0, 0, 0, 0):
                x, y, w, h = roi_box
                cropped = frame[y:y+h, x:x+w]
                cv2.imshow("Cropped ROI", cropped)
                print(f"ROI Coordinates: x={x}, y={y}, w={w}, h={h}")
            else:
                print("No ROI selected.")

        elif key == 27:  # ESC to exit
            break

    cap.release()
    cv2.destroyAllWindows()

    print(f"Final ROI for Camera {d}: {roi_box}")
    roi_boxes.append(roi_box)

# Save the ROI coordinates to a file
roi_file_path = os.path.join(calib_base_folder, f"roi_cam{d}.txt")
with open(roi_file_path, 'w') as f:
    for box in roi_boxes:
        f.write(f"{box[0]} {box[1]} {box[2]} {box[3]}\n")

Press SPACE to freeze frame and select ROI.
Draw the box, then press ENTER to confirm or ESC to cancel.
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
ROI Coordinates: x=152, y=119, w=328, h=176
(152, 119, 328, 176)


# Collect Intrinsic Calibration Images

In [ ]:
import cv2
import os
import subprocess

devices = [f"/dev/cam{i}" for i in [0, 1, 2]]

for i, device in enumerate(devices):
    print(f"Collecting instrinsic calibration images for camera {i}...")
    output_dir = f"{calib_base_folder}/charuco_calib_images/cam{i}"  # Directory to save images
    os.makedirs(output_dir, exist_ok=True)

    # Initialize webcam
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()

    print("Press SPACE to capture image, ESC to exit.")

    image_count = 0
    configure_wait_time = 2 # wait this many seconds to configure manual settings
    configure_yet = False
    start_time = time.time()
    while True:
        if time.time() - start_time > configure_wait_time and not configure_yet:
            configure_camera([device], config_commands)
            configure_yet = True
            print("Configured manual camera settings!!!")
        
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Display the live feed
        cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

        # Keyboard input
        key = cv2.waitKey(1) & 0xFF

        # Save image on SPACE key press
        if key == ord(' '):
            # Construct image filename
            image_path = os.path.join(output_dir, f"img_{image_count:03d}.png")
            # Save the frame as PNG
            cv2.imwrite(image_path, frame)
            print(f"Captured: {image_path}")
            image_count += 1

        # Exit on ESC key press
        elif key == 27:
            print("Exiting...")
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


# Collect Stereo Calibration Images

In [ ]:
# Collect images of calibration board in both cameras frames for stereo extrinsic calibration
import cv2
import datetime
import os

device_ids = [0, 1]
cur_devices = [f"/dev/cam{i}" for i in device_ids]
output_dir = f"{calib_base_folder}/stereo_calib_images"
os.makedirs(output_dir, exist_ok=True)

caps = []
for device in cur_devices:
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    if not cap.isOpened():
        print(f"Error: Camera {device} could not be opened.")
        exit()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)
    caps.append(cap)

configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera(cur_devices, config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    frames = []
    for cap in caps:
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame could not be read from camera.")
            exit()
        # resize for better display
        frame = cv2.resize(frame, (640, 480))
        frames.append(frame)

    # Combine and display both frames
    combined = cv2.hconcat(frames)
    cv2.imshow("Camera 0 (top) + Camera 1 (side)", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        for i, d in enumerate(device_ids):
            img_path = f"{output_dir}/cam{d}_{frame_count}.png"
            cv2.imwrite(img_path, frames[i])
            print(f"Captured images:\n - {img_path}\n")
        frame_count += 1

# Release both cameras and close windows
for cap in caps:
    cap.release()
cv2.destroyAllWindows()

# Collect Camera-World Calibration Images

In [ ]:
# Collect images for finding camera-world transform
# Current approach is to use aruco marker to find camera 0 pose wrt to aruco marker
# Then camera-camera transform can be used to find camera 1 to world transform
import time

# Make sure cameras are configured
# Open cameras with higher resolution
cap0 = cv2.VideoCapture(cam0_device, cv2.CAP_V4L2)
cap1 = cv2.VideoCapture(cam1_device, cv2.CAP_V4L2)
cap0.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap0.set(cv2.CAP_PROP_FRAME_HEIGHT, 920)
cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 920)

if not cap0.isOpened() or not cap1.isOpened():
    print("Error: One or both cameras could not be opened.")
    exit()

output_dir = f"{calib_base_folder}/extrinsic_calib_images"
os.makedirs(output_dir, exist_ok=True)
configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
frame_count = 0
while True:

    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera([cam0_device, cam1_device], config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frame from camera
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()

    if not ret0 or not ret1:
        print("Error: Frame could not be read.")
        break

    # Combine and display both frames
    combined = cv2.hconcat([frame0, frame1])
    cv2.imshow("Camera 0 (top) + Camera 1 (side)", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        img0_path = f"{output_dir}/cam0_{frame_count}.png"
        img1_path = f"{output_dir}/cam1_{frame_count}.png"
        cv2.imwrite(img0_path, frame0)
        cv2.imwrite(img1_path, frame1)
        print(f"Captured images:\n - {img0_path}\n - {img1_path}")
        frame_count += 1

# Release both cameras and close windows
cap0.release()
cap1.release()
cv2.destroyAllWindows()

# Compute Camera Calibration

## Compute Intrinsics Charuco

In [8]:
# Compute intrinsic, stereo extrinsic, and camera to world transforms
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle

def calibrate_intrinsics_charuco(glob_pattern, charuco_board, aruco_dict,
                                 min_markers=20):
    '''
    Calibrates camera intrinsics using a ChArUco board.
    '''
    all_corners, all_ids, img_size = [], [], None

    for fname in glob.glob(glob_pattern):
        img = cv2.imread(fname)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_size is None:
            img_size = img_gray.shape[::-1]

        detector = aruco.CharucoDetector(charuco_board)
        charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(img_gray)

        all_corners.append(charuco_corners)
        all_ids.append(charuco_ids)

    # 3. Calibrate
    ret, K, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=all_corners,
        charucoIds=all_ids,
        board=charuco_board,
        imageSize=img_size,
        cameraMatrix=None,
        distCoeffs=None
    )
    return ret, K, dist, rvecs, tvecs

# Dict holding all camera calibration data
camera_calib_data = {
        "cam0": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": { # this is camera to world transform
                "R": None,
                "T": None
            }
        },
        "cam1": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": {
                "R": None,
                "T": None
            }
        },
        "stereo_extrinsics": { # Camera to camera transform (from cam0 to cam1)
            "R_1_0": None,
            "T_1_0": None,
        },
    }

# Define ChArUco board used
square_size = 0.006
marker_length = 0.004
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

# Calibrate instrinsics
for i in range(2):
    intrinsic_calib_images_path = f"{calib_base_folder}/charuco_calib_images/cam{i}/*.png"
    rms_error, K, d, _, _ = calibrate_intrinsics_charuco(
        intrinsic_calib_images_path, charuco_board, aruco_dict
    )
    print(f"Charuco intrinsics for camera {i} RMS error: {rms_error:.4f}")
    print(f"Camera {i} intrinsic matrix:\n{K}")
    print(f"Camera {i} distortion coefficients:\n{d}")
    camera_calib_data[f"cam{i}"]["intrinsics"]["K"] = K
    camera_calib_data[f"cam{i}"]["intrinsics"]["d"] = d

print("Intrinsic calibration complete.")


NameError: name 'calib_base_folder' is not defined

## Compute Stereo Extrinsics Charuco

In [ ]:
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle

def detect_charuco_corners(image_path, board, detector):
    """Detects Charuco corners using OpenCV 4.11+ functions."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect Aruco markers and Charuco corners
    charuco_corners, charuco_ids, _, _ = detector.detectBoard(gray)

    if charuco_ids is not None and len(charuco_ids) > 4:
        return charuco_corners, charuco_ids
    return None, None

def calculate_relative_transform(R1, T1, R2, T2):
    """Calculate the relative rotation and translation between two cameras.
    Returns R21 and T21, the rotation and translation of camera 2 in camera 1's frame."""
    R21 = np.dot(R2, R1.T)
    T21 = T2 - np.dot(R21, T1)
    return R21, T21

def get_extrinsics(image_path, K, dist, board, detector):
    """Compute extrinsics for a single image."""
    charuco_corners, charuco_ids = detect_charuco_corners(image_path, board, detector)
    if charuco_corners is not None:
        obj_points = board.getChessboardCorners()[charuco_ids.flatten()]
        ret, rvec, tvec = cv2.solvePnP(obj_points, charuco_corners, K, dist)
        if ret:
            R, _ = cv2.Rodrigues(rvec)
            return R, tvec
    return None, None

# # Load intrinsic parameters
# calib_data_file = "../camera_calibration/05-08-25/camera_calib_data.pkl"
# with open(calib_data_file, 'rb') as f:
#     calib_data = pickle.load(f)

K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]

# Define ChArUco board used
square_size = 0.006
marker_length = 0.004
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

# Create the Aruco and Charuco detector objects (new in OpenCV 4.11+)
aruco_detector = aruco.ArucoDetector(aruco_dict)
charuco_detector = aruco.CharucoDetector(charuco_board)

# Paths to synchronized images
cam0_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cam0_*.png"))
cam1_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cam1_*.png"))

# Arrays to store transformations
relative_rotations = []
relative_translations = []

for img0_path, img1_path in zip(cam0_images, cam1_images):
    # Get extrinsics for both cameras using updated functions
    R0, T0 = get_extrinsics(img0_path, K0, dist0, charuco_board, charuco_detector)
    R1, T1 = get_extrinsics(img1_path, K1, dist1, charuco_board, charuco_detector)

    if R0 is not None and R1 is not None:
        # Calculate the relative transformation between cameras
        R10, T10 = calculate_relative_transform(R0, T0, R1, T1)
        relative_rotations.append(R10)
        relative_translations.append(T10)

# Average the transformations
R_avg = sum(relative_rotations) / len(relative_rotations)
T_avg = sum(relative_translations) / len(relative_translations)

# Print the averaged relative transformation
print("Transform from Camera 0 to Camera 1:")
print("Averaged Relative Rotation (R10):\n", R_avg)
print("Averaged Relative Translation (T10):\n", T_avg)

# Save the relative transformation
camera_calib_data["stereo_extrinsics"]["R_1_0"] = R_avg
camera_calib_data["stereo_extrinsics"]["T_1_0"] = T_avg


## Compute Camera-World Transform

In [1]:
# Find camera to world using PnP and calibrator piece
import cv2
import os

img_dir = "../camera_calibration/05-16-25/extrinsic_calib_images"
img0 = cv2.imread(os.path.join(img_dir, "cam0_0.png"))
img1 = cv2.imread(os.path.join(img_dir, "cam1_0.png"))
# Downsize images for display
img0 = cv2.resize(img0, (640, 480))
img1 = cv2.resize(img1, (640, 480))


In [2]:

# Lists to store clicked points
points_img0 = []
points_img1 = []

def click_event_img0(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points_img0.append((x, y))
        print(f"Image 0: Point {len(points_img0)}: ({x}, {y})")
        cv2.circle(img0, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Cam0", img0)

def click_event_img1(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points_img1.append((x, y))
        print(f"Image 1: Point {len(points_img1)}: ({x}, {y})")
        cv2.circle(img1, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Cam1", img1)

cv2.imshow("Cam0", img0)
cv2.setMouseCallback("Cam0", click_event_img0)
cv2.imshow("Cam1", img1)
cv2.setMouseCallback("Cam1", click_event_img1)

print("Click points in both images. Press any key to finish and close windows.")
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Points in img0:", points_img0)
print("Points in img1:", points_img1)


Click points in both images. Press any key to finish and close windows.
Points in img0: []
Points in img1: []


In [3]:
# Read in camera intrinsics
with open("../camera_calibration/05-08-25/camera_calib_data.pkl", 'rb') as f:
    camera_calib_data = pickle.load(f)
K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]

# Defined corresponding world frame coordinates of clicked points
# These should be in the same order as the clicked points
world_coords_0 = np.array([[0, 4.125, 0],
                [0, 16, 9],
                [0, 0, 23.95],
                [-4.125, 0, 0],
                [-16, 0, 9],
                [4.125, 0, 0],
                [16, 0, 9]]) / 1000

world_coords_1 = np.array([[0, 4.125, 0],
                [0, 16, 9],
                [0, 0, 23.95],
                [-4.125, 0, 0],
                [-16, 0, 9],
                [0, -4.125, 0],
                [0, -16, 9]]) / 1000

# Find world frame transform for each camera using solvePnP
def find_camera_to_world_transform(points_img, world_coords, K, dist):
    # Convert points to numpy arrays
    points_img = np.array(points_img, dtype=np.float32)
    world_coords = np.array(world_coords, dtype=np.float32)

    # Solve PnP
    _, rvec, tvec = cv2.solvePnP(world_coords, points_img, K, dist)
    R, _ = cv2.Rodrigues(rvec)
    return R, tvec

R0, T0 = find_camera_to_world_transform(points_img0, world_coords_0, K0, dist0)
R1, T1 = find_camera_to_world_transform(points_img1, world_coords_1, K1, dist1)
print("Camera 0 to world transform:")
print("Rotation:\n", R0)
print("Translation:\n", T0)
print("Camera 1 to world transform:")
print("Rotation:\n", R1)
print("Translation:\n", T1)

# Visualize world frame axes in each camera frame
def draw_axes(image, R, T, K, dist, axis_length=0.02):
    """Draw the world frame axes on the image."""
    axis_points = np.float32([
        [0, 0, 0],                  # Origin
        [axis_length, 0, 0],        # X-axis (red)
        [0, axis_length, 0],        # Y-axis (green)
        [0, 0, axis_length]         # Z-axis (blue)
    ]).reshape(-1, 3)
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


# # Replace extrinsic calibration parameters
# camera_calib_data["cam0"]["extrinsics"]["R"] = R0
# camera_calib_data["cam0"]["extrinsics"]["T"] = T0
# camera_calib_data["cam1"]["extrinsics"]["R"] = R1
# camera_calib_data["cam1"]["extrinsics"]["T"] = T1
# # Save the camera calibration data
# with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", 'wb') as f:
#     pickle.dump(camera_calib_data, f)

# Show camera 0 world frame transform in both camera frames
# Compute Camera 1 to world transform using stereo extrinsics
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"]
R1_from0 = np.dot(R_1_0, R0)
T1_from0 = np.dot(R_1_0, T0) + T_1_0
print("Camera 1 to world transform using stereo extrinsics:")
print("Rotation:\n", R1_from0)
print("Translation:\n", T1_from0)
# Show camera 1 world frame transform in both camera frames
R0_from1 = np.dot(R_1_0.T, R1)
T0_from1 = np.dot(R_1_0.T, T1) - T_1_0
print("Camera 0 to world transform using stereo extrinsics:")
print("Rotation:\n", R0_from1)
print("Translation:\n", T0_from1)

# Draw axes on img0 and img1
# draw_axes(img0, R0, T0, K0, dist0)
draw_axes(img0, R0_from1, T0_from1, K0, dist0)
cv2.imshow("Camera 0 - World Frame Axes from cam 0 transform", img0)
# cv2.imshow("Camera 1 - World Frame Axes from cam 0 transform", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Draw axes on img0 and img1
# draw_axes(img1, R1_from0, T1_from0, K1, dist1)
draw_axes(img1, R1, T1, K1, dist1)
# cv2.imshow("Camera 0 - World Frame Axes from cam 1 transform", img0)
cv2.imshow("Camera 1 - World Frame Axes from cam 1 transform", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'pickle' is not defined

In [ ]:
import numpy as np

def transform_matrix(R, t):
    """Construct a homogeneous transformation matrix from rotation and translation."""
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def check_transform_equivalence(R1, t1, R2, t2):
    """Check if two transformations are equivalent."""
    # Construct transformation matrices
    T1 = transform_matrix(R1, t1)
    T2 = transform_matrix(R2, t2)
    
    # Compute the relative transformation
    relative_transform = np.linalg.inv(T1) @ T2
    identity_matrix = np.eye(4)
    
    # Compute the deviation from the identity matrix
    deviation = np.linalg.norm(relative_transform - identity_matrix)

    # Print the relative transformation and the deviation
    print("Relative Transformation:\n", relative_transform)
    print("Deviation from Identity:", deviation)

    # Threshold for considering them equivalent
    tolerance = 1e-6
    if deviation < tolerance:
        print("The transformations are approximately equivalent.")
    else:
        print("The transformations are not equivalent.")

# # Example usage
# R1 = np.array([[-0.996, -0.087, 0.012],
#                [-0.032, 0.234, -0.972],
#                [0.082, -0.968, -0.236]])

# t1 = np.array([-0.003, 0.045, -0.221])

# R2 = np.array([[-0.104, -0.994, 0.029],
#                [-0.080, -0.020, -0.997],
#                [0.991, -0.106, -0.078]])

# t2 = np.array([0.015, 0.028, -0.273])

check_transform_equivalence(R1, T0.flatten(), R0_from1, T0_from1.flatten())
check_transform_equivalence(R1, T1.flatten(), R1_from0, T1_from0.flatten())


Relative Transformation:
 [[ 0.00558221  0.99967035  0.02471735  0.02573521]
 [-0.99877942  0.00438786  0.04866734  0.44270566]
 [ 0.04854917 -0.02497695  0.99848393  0.03046309]
 [ 0.          0.          0.          1.        ]]
Deviation from Identity: 2.044639980180462
The transformations are not equivalent.
Relative Transformation:
 [[-0.98008459 -0.19790326  0.01473119  0.45478905]
 [ 0.19204885 -0.96450462 -0.18119603 -0.42941468]
 [ 0.05005939 -0.17474519  0.98331469 -0.04832565]
 [ 0.          0.          0.          1.        ]]
Deviation from Identity: 2.8837469498880535
The transformations are not equivalent.


In [ ]:
import numpy as np
from scipy.optimize import least_squares
import pdb

### Currently does not work

def rotation_matrix_to_vector(R):
    """Convert a rotation matrix to a rotation vector using Rodrigues' formula."""
    rvec, _ = cv2.Rodrigues(R)
    return rvec.flatten()


def vector_to_rotation_matrix(rvec):
    """Convert a rotation vector to a rotation matrix using Rodrigues' formula."""
    R, _ = cv2.Rodrigues(rvec)
    return R


def transform_matrix(R, t):
    """Construct a homogeneous transformation matrix from rotation and translation."""
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T


def decompose_transform(T):
    """Decompose a transformation matrix into rotation and translation."""
    R = T[:3, :3]
    t = T[:3, 3]
    return R, t


def cost_function(params, R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W):
    """Cost function to minimize the difference between expected and actual transforms."""
    rvec0, rvec1, t0, t1 = np.split(params, [3, 6, 9])
    R0 = vector_to_rotation_matrix(rvec0)
    R1 = vector_to_rotation_matrix(rvec1)
    T0 = transform_matrix(R0, t0)
    T1 = transform_matrix(R1, t1)
    expected_T1 = T0 @ transform_matrix(R_1_0, T_1_0)
    residual = (T1 - expected_T1).flatten()
    return residual


def refine_transforms(R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W):
    """Optimize transforms to improve consistency."""
    rvec0 = rotation_matrix_to_vector(R_0_w)
    rvec1 = rotation_matrix_to_vector(R_1_w)
    # pdb.set_trace()
    initial_params = np.hstack([rvec0, rvec1, T_0_W, T_1_W])
    result = least_squares(cost_function, initial_params, args=(R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W))
    rvec0_opt, rvec1_opt, t0_opt, t1_opt = np.split(result.x, [3, 6, 9])
    R0_opt = vector_to_rotation_matrix(rvec0_opt)
    R1_opt = vector_to_rotation_matrix(rvec1_opt)
    return R0_opt, t0_opt, R1_opt, t1_opt

# Load the camera calibration data
with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", 'rb') as f:
    camera_calib_data = pickle.load(f)

R_0_w = camera_calib_data["cam0"]["extrinsics"]["R"]
T_0_W = camera_calib_data["cam0"]["extrinsics"]["T"].flatten()
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"].flatten()
R_1_w = camera_calib_data["cam1"]["extrinsics"]["R"]
T_1_W = camera_calib_data["cam1"]["extrinsics"]["T"].flatten()
# Refine the transforms
R0_refined, T0_refined, R1_refined, T1_refined = refine_transforms(R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W)
print("Refined R_0_w:\n", R0_refined)
print("Refined T_0_W:\n", T0_refined)
print("Refined R_1_w:\n", R1_refined)
print("Refined T_1_W:\n", T1_refined)


Refined R_0_w:
 [[-0.72882985  0.22494091  0.64669053]
 [-0.67096253 -0.42285115 -0.60910277]
 [ 0.1364417  -0.87783739  0.45911347]]
Refined T_0_W:
 [ 0.21338508 -0.74842295  0.92909777]
Refined R_1_w:
 [[ 0.63000373  0.2387844   0.73897044]
 [-0.58767338 -0.47546573  0.65465436]
 [ 0.50767637 -0.84670795 -0.15921795]]
Refined T_1_W:
 [ 0.23905185 -1.00647314  1.04948871]


### Aruco marker world frame calibration

#

In [3]:
import cv2
import cv2.aruco as aruco
import numpy as np
import pickle

def solve_pnp(marker_corners, marker_length, K, dist):
    """Solve PnP for a single Aruco marker."""
    half_length = marker_length / 2

    # 3D points of the marker corners in the world coordinate system (origin at one corner)
    obj_points = np.array([
        [0, 0, 0],                  # Bottom-left (origin)
        [marker_length, 0, 0],      # Bottom-right (X-axis)
        [marker_length, marker_length, 0], # Top-right
        [0, marker_length, 0]       # Top-left (Z-axis)
    ], dtype=np.float32)
    print("Object points:\n", obj_points)
    # Solve PnP to find rotation and translation
    ret, rvec, tvec = cv2.solvePnP(obj_points, marker_corners, K, dist)
    if ret:
        R, _ = cv2.Rodrigues(rvec)
        return R, tvec
    return None, None

def draw_axes(image, R, T, K, dist):
    """Draw the world frame origin and axes on the image."""
    axis_length = 0.05  # 5 cm

    # Define the 3D points for the axes
    axis_points = np.float32([
        [0, 0, 0],              # Origin
        [axis_length, 0, 0],    # X-axis (red)
        [0, axis_length, 0],    # Y-axis (green)
        [0, 0, axis_length]     # Z-axis (blue)
    ]).reshape(-1, 3)

    # Project the 3D axis points onto the image plane
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)

    # Draw the origin point
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.circle(image, origin, 5, (0, 0, 255), -1)

    # Draw the X, Y, Z axes
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)  # X-axis (red)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)  # Y-axis (green)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)  # Z-axis (blue)

    # Annotate the axes
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


# Load intrinsic calibration data
# # Load intrinsic parameters
calib_data_file = f"../camera_calibration/05-08-25/camera_calib_data.pkl"
with open(calib_data_file, 'rb') as f:
    camera_calib_data = pickle.load(f)

K = camera_calib_data["cam0"]["intrinsics"]["K"]
dist = camera_calib_data["cam0"]["intrinsics"]["d"]


# Aruco marker parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
marker_length = 0.004  # Length of the marker side in meters

# Initialize the Aruco detector
detector = aruco.ArucoDetector(aruco_dict)

# Load image
img = cv2.imread(f"../camera_calibration/05-08-25/extrinsic_calib_images/cam0/img_aruco.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect the Aruco marker
corners, ids, _ = detector.detectMarkers(img_gray)
print(corners)

if ids is not None:
    R_0_w, T_0_w = solve_pnp(corners[0], marker_length, K, dist)
    if R_0_w is not None:
        print(f"Camera 0 to World Rotation:\n{R_0_w}")
        print(f"Camera 0 to World Translation:\n{T_0_w}")

        # Draw the marker and the world frame
        aruco.drawDetectedMarkers(img, corners, ids)
        draw_axes(img, R_0_w, T_0_w, K, dist)

# Compute Camera 1 to world transform using stereo extrinsics
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"]
R_1_w = np.dot(R_1_0, R_0_w)
T_1_w = np.dot(R_1_0, T_0_w) + T_1_0
print(f"Camera 1 to World Rotation:\n{R_1_w}")
print(f"Camera 1 to World Translation:\n{T_1_w}")

# save all calibration data
camera_calib_data["cam0"]["extrinsics"]["R"] = R_0_w
camera_calib_data["cam0"]["extrinsics"]["T"] = T_0_w
camera_calib_data["cam1"]["extrinsics"]["R"] = R_1_w
camera_calib_data["cam1"]["extrinsics"]["T"] = T_1_w

# with open(f"{calib_base_folder}/camera_calib_data.pkl", "wb") as f:
#     pickle.dump(camera_calib_data, f)

cv2.imshow("Camera 0 - Aruco Marker with World Frame", img)
# cv2.imwrite(f"{calib_base_folder}/extrinsic_calib_images/cam0/img_aruco_with_axes.png", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(array([[[295., 147.],
        [307., 146.],
        [308., 159.],
        [296., 159.]]], dtype=float32),)
Object points:
 [[0.    0.    0.   ]
 [0.004 0.    0.   ]
 [0.004 0.004 0.   ]
 [0.    0.004 0.   ]]
Camera 0 to World Rotation:
[[ 0.96241617  0.07780901  0.26019391]
 [ 0.002285    0.95572452 -0.29425401]
 [-0.27156931  0.28378936  0.91962694]]
Camera 0 to World Translation:
[[-0.00391395]
 [-0.03319903]
 [ 0.20169472]]
Camera 1 to World Rotation:
[[ 0.27576602 -0.25516868 -0.92672725]
 [-0.05900692  0.95778878 -0.28126257]
 [ 0.95939296  0.13223629  0.24905744]]
Camera 1 to World Translation:
[[-0.03173681]
 [-0.01782421]
 [ 0.22452266]]


In [35]:
# Verify camera to world transform by projecting axes onto image
import cv2
import numpy as np
import cv2.aruco as aruco
import pickle
import os

# Load calibration data
with open(f"../camera_calibration/05-08-25/camera_calib_data.pkl", "rb") as f:
    camera_calib_data = pickle.load(f)

# Load intrinsic and extrinsic parameters for both cameras
K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
R_0_w = camera_calib_data["cam0"]["extrinsics"]["R"]
T_0_w = camera_calib_data["cam0"]["extrinsics"]["T"]
print("Camera 0 to World Rotation:\n", R_0_w)
print("Camera 0 to World Translation:\n", T_0_w)

K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]
R_1_w = camera_calib_data["cam1"]["extrinsics"]["R"]
T_1_w = camera_calib_data["cam1"]["extrinsics"]["T"]
print("Camera 1 to World Rotation:\n", R_1_w)
print("Camera 1 to World Translation:\n", T_1_w)

# Load images for verification
img0 = cv2.imread(f"../tip_pose_images/cam0_0.png")
img1 = cv2.imread(f"../tip_pose_images/cam1_0.png")

# Draw world frame axes on Camera 0 image
draw_axes(img0, R_0_w, T_0_w, K0, dist0)
cv2.imshow("Camera 0 - World Frame Projection", img0)

# Draw world frame axes on Camera 1 image
draw_axes(img1, R_1_w, T_1_w, K1, dist1)
cv2.imshow("Camera 1 - World Frame Projection", img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

Camera 0 to World Rotation:
 [[ 0.96241617  0.07780901  0.26019391]
 [ 0.002285    0.95572452 -0.29425401]
 [-0.27156931  0.28378936  0.91962694]]
Camera 0 to World Translation:
 [[-0.00391395]
 [-0.03319903]
 [ 0.20169472]]
Camera 1 to World Rotation:
 [[ 0.27576602 -0.25516868 -0.92672725]
 [-0.05900692  0.95778878 -0.28126257]
 [ 0.95939296  0.13223629  0.24905744]]
Camera 1 to World Translation:
 [[-0.03173681]
 [-0.01782421]
 [ 0.22452266]]


# Test

In [7]:
import pickle
import numpy as np

with open("../camera_calibration/05-08-25/camera_calib_data.pkl", "rb") as f:
    camera_calib = pickle.load(f)

K0 = camera_calib["cam0"]["intrinsics"]["K"]
dist0 = camera_calib["cam0"]["intrinsics"]["d"]
R_1_0 = camera_calib["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib["stereo_extrinsics"]["T_1_0"]

img_points = np.array([
    [317., 117.],
    [318., 144.],
    [316., 199.],
    [308., 118.],
    # [278, 148],
    # [326, 118],
    # [354, 150]
], dtype=np.float32)
obj_points = 0.001 * np.array([[0, 4.125, 0],
                [0, 16, 9],
                [0, 0, 23.95],
                [-4.125, 0, 0],
                # [-16, 0, 9],
                # [4.125, 0, 0],
                # [16, 0, 9]
                ])

ret, rvec, tvec, _ = cv2.solvePnPRansac(obj_points, img_points, K0, dist0, flags=cv2.SOLVEPNP_P3P)
R0 = cv2.Rodrigues(rvec)[0]
T0 = tvec
R1 = np.dot(R_1_0, R0)
T1 = np.dot(R_1_0, T0) + T_1_0
print("R1:\n", R1)
print("T1:\n", T1)

# Update and save camera calibration data
camera_calib["cam0"]["extrinsics"]["R"] = R0
camera_calib["cam0"]["extrinsics"]["T"] = T0
camera_calib["cam1"]["extrinsics"]["R"] = R1
camera_calib["cam1"]["extrinsics"]["T"] = T1

with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", "wb") as f:
    pickle.dump(camera_calib, f)

R1:
 [[-0.00941198  0.99909794 -0.04118328]
 [-0.0493389   0.04065417  0.99792921]
 [ 0.99871333  0.01144134  0.04890463]]
T1:
 [[-0.01961234]
 [-0.02486894]
 [ 0.2308288 ]]


In [9]:
# Visualize new transform
import cv2
import numpy as np
import pickle

def draw_axes(image, R, T, K, dist):
    """Draw the world frame origin and axes on the image."""
    axis_length = 0.05  # 5 cm

    # Define the 3D points for the axes
    axis_points = np.float32([
        [0, 0, 0],              # Origin
        [axis_length, 0, 0],    # X-axis (red)
        [0, axis_length, 0],    # Y-axis (green)
        [0, 0, axis_length]     # Z-axis (blue)
    ]).reshape(-1, 3)

    # Project the 3D axis points onto the image plane
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)

    # Draw the origin point
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.circle(image, origin, 5, (0, 0, 255), -1)

    # Draw the X, Y, Z axes
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)  # X-axis (red)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)  # Y-axis (green)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)  # Z-axis (blue)

    # Annotate the axes
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", "rb") as f:
    camera_calib = pickle.load(f)

K0 = camera_calib["cam0"]["intrinsics"]["K"]
dist0 = camera_calib["cam0"]["intrinsics"]["d"]
R0 = camera_calib["cam0"]["extrinsics"]["R"]
T0 = camera_calib["cam0"]["extrinsics"]["T"]
K1 = camera_calib["cam1"]["intrinsics"]["K"]
dist1 = camera_calib["cam1"]["intrinsics"]["d"]
R1 = camera_calib["cam1"]["extrinsics"]["R"]
T1 = camera_calib["cam1"]["extrinsics"]["T"]

# Load images for verification
img0 = cv2.imread("../camera_calibration/05-16-25/extrinsic_calib_images/cam0_0.png")
img1 = cv2.imread("../camera_calibration/05-16-25/extrinsic_calib_images/cam1_0.png")
# Downsize images for display
img0 = cv2.resize(img0, (640, 480))
img1 = cv2.resize(img1, (640, 480))

# Draw world frame axes on Camera 0 image
draw_axes(img0, R0, T0, K0, dist0)
cv2.imshow("Camera 0 - World Frame Projection", img0)
# Draw world frame axes on Camera 1 image
draw_axes(img1, R1, T1, K1, dist1)
cv2.imshow("Camera 1 - World Frame Projection", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Write calibration matrices to python module

In [10]:
import numpy as np
import pickle

# Load camera calibration data from pkl file
with open("../camera_calibration/05-16-25/camera_calib_data.pkl", "rb") as f:
    camera_calib_data = pickle.load(f)

# Write to a python module
calib_module_path = "camera_calib_data.py"
with open(calib_module_path, "w") as f:
    f.write("# This file is auto-generated from camera calibration data.\n")
    f.write("import numpy as np\n\n")
    f.write(f"K0 = np.{repr(camera_calib_data['cam0']['intrinsics']['K'])}\n")
    f.write(f"d0 = np.{repr(camera_calib_data['cam0']['intrinsics']['d'])}\n")
    f.write(f"K1 = np.{repr(camera_calib_data['cam1']['intrinsics']['K'])}\n")
    f.write(f"d1 = np.{repr(camera_calib_data['cam1']['intrinsics']['d'])}\n")
    f.write(f"R_1_0 = np.{repr(camera_calib_data['stereo_extrinsics']['R_1_0'])}\n")
    f.write(f"T_1_0 = np.{repr(camera_calib_data['stereo_extrinsics']['T_1_0'])}\n")
    f.write(f"R_0_w = np.{repr(camera_calib_data['cam0']['extrinsics']['R'])}\n")
    f.write(f"T_0_w = np.{repr(camera_calib_data['cam0']['extrinsics']['T'])}\n")
    f.write(f"R_1_w = np.{repr(camera_calib_data['cam1']['extrinsics']['R'])}\n")
    f.write(f"T_1_w = np.{repr(camera_calib_data['cam1']['extrinsics']['T'])}\n")


In [9]:
from camera_calib_data import *

print(K0)

[[629.50286944   0.         307.21941529]
 [  0.         630.77155779 250.59295032]
 [  0.           0.           1.        ]]
